In [ ]:
import os
from binance.client import Client
from binance.enums import *
import pandas as pd
from time import sleep


api_key = 'api key'
api_secret = 'secret key'


client = Client(api_key, api_secret)


In [ ]:

symbol = 'BTCUSDT'  
investment_amount = 100  
interval = 60 * 60 * 24  
total_investment = 1000  
price_drop_threshold = 0.08


In [ ]:

def get_current_price(symbol):
    avg_price = client.get_avg_price(symbol=symbol)
    return float(avg_price['price'])


def place_market_order(symbol, quantity):
    try:
        order = client.futures_create_order(
            symbol=symbol,
            side=SIDE_BUY,
            type=ORDER_TYPE_MARKET,
            quantity=quantity
        )
        return order
    except Exception as e:
        print(f"An exception occurred: {e}")
        return None

def calculate_quantity(investment_amount, current_price):
    return round(investment_amount / current_price, 6)

def dca_bot(symbol, initial_investment_amount, interval, total_investment, price_drop_threshold):
    invested_amount = 0
    investment_amount = initial_investment_amount
    initial_price = get_current_price(symbol)
    last_price = initial_price

    while invested_amount < total_investment:
        current_price = get_current_price(symbol)

        
        if current_price <= last_price * (1 - price_drop_threshold):
            investment_amount *= 2
            last_price = current_price  

        quantity = calculate_quantity(investment_amount, current_price)
        print(f"Placing order for {quantity} {symbol} at price {current_price}")
        order = place_market_order(symbol, quantity)
        
        if order:
            invested_amount += investment_amount
            print(f"Total invested: {invested_amount} USD")
        else:
            print("Order failed, retrying after interval")
        
        sleep(interval)
    
    print("DCA completed")




In [ ]:
#Starting the bot
dca_bot(symbol, investment_amount, interval, total_investment,price_drop_threshold)